# Dependencies

In [1]:
pip install mlxtend

In [28]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
df = pd.read_csv("Groceries_dataset.csv")
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(38765, 3)

In [5]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


# Data Cleaning

Steps:

* Adjust data type;
* Rename column;

In [6]:
df.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [7]:
df['Date'] = pd.to_datetime(df['Date'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-7-b9c6864122be>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])


In [8]:
df['Member_number'] = df['Member_number'].astype(str)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df.rename(columns={"Member_number":"customerId"}, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df.tail()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customerId,Date,itemDescription
38760,4471,2014-08-10,sliced cheese
38761,2022,2014-02-23,candy
38762,1097,2014-04-16,cake bar
38763,1510,2014-03-12,fruit/vegetable juice
38764,1521,2014-12-26,cat food


# Exploratory Data Analysis

Steps:

* Detecting Weekday;
* Detecting Month of Year;

In [11]:
df['weekday'] = df['Date'].dt.weekday
df['weekday'] = df['weekday'].replace((0,1,2,3,4,5,6),("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
df['itemDescription'] = df['itemDescription'].str.strip()
df['itemDescription'] = df['itemDescription'].str.capitalize()
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customerId,Date,itemDescription,weekday
0,1808,2015-07-21,Tropical fruit,Tuesday
1,2552,2015-05-01,Whole milk,Friday
2,2300,2015-09-19,Pip fruit,Saturday
3,1187,2015-12-12,Other vegetables,Saturday
4,3037,2015-01-02,Whole milk,Friday
...,...,...,...,...
38760,4471,2014-08-10,Sliced cheese,Sunday
38761,2022,2014-02-23,Candy,Sunday
38762,1097,2014-04-16,Cake bar,Wednesday
38763,1510,2014-03-12,Fruit/vegetable juice,Wednesday


# Data Viz

ASAP

# Data Preprocessing

Steps:

* Group by;
* Make a matrice;

In [13]:
transactions = df.groupby(["customerId","itemDescription"])["itemDescription"].count().reset_index(name='count')
transactions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customerId,itemDescription,count
0,1000,Canned beer,1
1,1000,Hygiene articles,1
2,1000,Misc. beverages,1
3,1000,Pastry,1
4,1000,Pickled vegetables,1
...,...,...,...
34761,5000,Onions,1
34762,5000,Other vegetables,1
34763,5000,Root vegetables,1
34764,5000,Semi-finished bread,1


In [14]:
# making a mxn matrice where m=transaction and n=items and each row represents whether the item was in the transaction or not
basket = transactions.pivot_table(index='customerId',columns='itemDescription',values='count',aggfunc='size').fillna(0)
basket = basket.astype(int)
basket.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


itemDescription,Abrasive cleaner,Artif. sweetener,Baby cosmetics,Bags,Baking powder,Bathroom cleaner,Beef,Berries,Beverages,Bottled beer,...,Uht-milk,Vinegar,Waffles,Whipped/sour cream,Whisky,White bread,White wine,Whole milk,Yogurt,Zwieback
customerId,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1001,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,1,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Model Building

* Use Apriori Algorithm with min_support representind 1% of transactions;
* Make the rules of association from frequent itens;

In [15]:
frequent_items = apriori(basket, min_support = 0.01,use_colnames = True)
frequent_items

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.031042,(Baking powder)
1,0.119548,(Beef)
2,0.079785,(Berries)
3,0.062083,(Beverages)
4,0.158799,(Bottled beer)
...,...,...
3011,0.011031,"(Whipped/sour cream, Yogurt, Soda, Whole milk)"
3012,0.010518,"(Whole milk, Yogurt, Other vegetables, Rolls/b..."
3013,0.013597,"(Whole milk, Yogurt, Other vegetables, Rolls/b..."
3014,0.010005,"(Whole milk, Shopping bags, Yogurt, Other vege..."


In [16]:
rules = association_rules(frequent_items, metric = "lift", min_threshold = 1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6938,"(Domestic eggs, Meat)",(Whole milk),0.013084,0.458184,0.010262,0.784314,1.711789,0.004267,2.512057,0.421328
5626,"(Fruit/vegetable juice, Chocolate)",(Whole milk),0.014366,0.458184,0.010775,0.750000,1.636898,0.004192,2.167265,0.394760
15144,"(Other vegetables, Yogurt, Bottled water, Roll...",(Whole milk),0.014110,0.458184,0.010518,0.745455,1.626978,0.004053,2.128564,0.390879
11619,"(Yogurt, Bottled water, Pip fruit)",(Whole milk),0.013853,0.458184,0.010262,0.740741,1.616689,0.003914,2.089863,0.386811
11983,"(Yogurt, Rolls/buns, Brown bread)",(Whole milk),0.017445,0.458184,0.012827,0.735294,1.604802,0.004834,2.046862,0.383561
...,...,...,...,...,...,...,...,...,...,...
3633,(Whole milk),"(Bottled water, Meat)",0.458184,0.015136,0.010005,0.021837,1.442690,0.003070,1.006850,0.566336
4931,(Whole milk),"(Chocolate, Canned beer)",0.458184,0.017188,0.010005,0.021837,1.270428,0.002130,1.004752,0.392871
6247,(Whole milk),"(White bread, Citrus fruit)",0.458184,0.018984,0.010005,0.021837,1.150253,0.001307,1.002916,0.241089
15225,(Whole milk),"(Other vegetables, Shopping bags, Rolls/buns, ...",0.458184,0.014110,0.010005,0.021837,1.547613,0.003540,1.007899,0.653069


In [17]:
# arranging the data from highest to lowest with respect to 'confidence'

rules.sort_values('confidence', ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6938,"(Domestic eggs, Meat)",(Whole milk),0.013084,0.458184,0.010262,0.784314,1.711789,0.004267,2.512057,0.421328
5626,"(Fruit/vegetable juice, Chocolate)",(Whole milk),0.014366,0.458184,0.010775,0.750000,1.636898,0.004192,2.167265,0.394760
15144,"(Other vegetables, Yogurt, Bottled water, Roll...",(Whole milk),0.014110,0.458184,0.010518,0.745455,1.626978,0.004053,2.128564,0.390879
11619,"(Yogurt, Bottled water, Pip fruit)",(Whole milk),0.013853,0.458184,0.010262,0.740741,1.616689,0.003914,2.089863,0.386811
11983,"(Yogurt, Rolls/buns, Brown bread)",(Whole milk),0.017445,0.458184,0.012827,0.735294,1.604802,0.004834,2.046862,0.383561
...,...,...,...,...,...,...,...,...,...,...
6847,(Whole milk),"(Pastry, Dessert)",0.458184,0.018728,0.010005,0.021837,1.166010,0.001424,1.003178,0.262772
15024,(Whole milk),"(Shopping bags, Sausage, Yogurt)",0.458184,0.015393,0.010005,0.021837,1.418645,0.002953,1.006588,0.544653
10898,(Whole milk),"(Bottled water, Soda, Bottled beer)",0.458184,0.016932,0.010005,0.021837,1.289677,0.002247,1.005014,0.414554
8521,(Whole milk),"(Sausage, Napkins)",0.458184,0.018984,0.010005,0.021837,1.150253,0.001307,1.002916,0.241089
